In [2]:
import pandas as pd
from datetime import datetime as dt
import ujson as js

from ApiHandler import ApiHandler

In [3]:
class ApiServerStatus(ApiHandler):
    settings = {
        **ApiHandler.settings,
    }
    url = {
        **ApiHandler.url,
        'data': 'https://esi.evetech.net/latest/status/',
    }
    script_vals = {
        **ApiHandler.script_vals,
        'table': 'ServerStatus'
    }
    name = 'ESI Server Status API'
    
    def build_data(self, raw_data):
        if self.verbose: self._verbose('build_data', 'Building data series...')
        
        data_series = pd.Series(js.loads(raw_data))
        data_series['record_time'] = self.timestamp
        data_series['start_time'] = dt.strptime(data_series['start_time'], '%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d %H:%M:%S')
        
        if self.verbose: self._verbose('build_data', 'Data series built.')
            
        return data_series
            
    def data_inserter(self, data_series):
        if self.verbose: self._verbose('data_inserter', 'Inserting record...')
            
        insert_script = self.sql['insert'].format(**{
            'table': self.script_vals['table'],
            'cols': ','.join(data_series.index),
            'vals': ','.join(['%s']*data_series.index.size),
            'upsert': ','.join(['{col}=VALUES({col})'.format(col=col) for col in data_series.index])
        })
        
        cur = self.conn['maria'].cursor()
        cur.execute(insert_script, data_series.values.tolist())
        self.conn['maria'].commit()
        cur.close()
        
        if self.verbose: self._verbose('data_inserter', 'Record inserted.')

In [4]:
api = ApiServerStatus()
api.run_process()